# Attribution Models

## What Are Attribution Models?

Attribution models are methods used in marketing to **assign credit** to
the various channels or touchpoints that contribute to a customer’s
conversion (e.g., making a purchase or signing up for a service).
Consider these simple examples:

-   **First-Touch Attribution:**  
    *Example:* A customer discovers your brand through an Instagram post
    and later makes a purchase. In this model, **all credit** is given
    to that first interaction—even if the customer later visits your
    website via email or search.

-   **Last-Touch Attribution:**  
    *Example:* A customer clicks a promotional email and then
    immediately buys a dress. Here, **all the credit** goes to the email
    interaction, ignoring any earlier exposures through social media or
    other channels.

-   **Multi-Touch Attribution:**  
    *Example:* A customer sees an influencer post on Instagram, clicks
    on a retargeting ad, and finally converts after receiving an email
    about a flash sale. This model distributes the credit **across all
    channels**, reflecting the reality that each interaction contributed
    to the final purchase decision.

## Case Study: LuxeStyle Boutique

Let’s examine data from LuxeStyle Boutique, a direct-to-consumer fashion
brand that specializes in contemporary women’s clothing and accessories.
The company has both an online presence and several physical stores,
using multiple marketing channels to reach their fashion-forward target
audience. They’re particularly known for their sustainable practices and
size-inclusive collections.

LuxeStyle’s marketing mix includes:

-   Email campaigns featuring new collections, styling tips, and
    exclusive pre-sales
-   Strong social media presence on Instagram, Pinterest, and TikTok
-   Paid search advertising focusing on fashion-related keywords
-   Organic search optimization for style guides and trend content
-   Direct traffic from brand recognition and offline advertising
-   Influencer partnerships (tracked through direct traffic and special
    URLs)

## How Is This Data Collected?

Before diving into our analysis, it’s important to understand how
fashion e-commerce companies actually collect this channel exposure
data. In our simulation, we’ll generate 1s and 0s to represent channel
exposure, but in reality, this data comes from multiple tracking systems
and tools.

### Channel Tracking Methods

Each marketing channel has specific methods for tracking customer
exposure:

1.  **Email (0/1)**
    -   1 = Customer opened email (tracked via pixel) or clicked link
        (tracked via UTM)
    -   Example: Customer clicks “Shop Now” in a new collection email
    -   Tracked through: Klaviyo, Mailchimp
2.  **Social Media (0/1)**
    -   1 = Click-through from social platforms or social-specific promo
        code use
    -   Example: Customer taps Instagram shopping tag and visits site
    -   Tracked through: Facebook Pixel, Platform Analytics
3.  **Paid Search (0/1)**
    -   1 = Click on Google/Bing ad
    -   Example: Customer clicks shopping ad for “summer dresses”
    -   Tracked through: Google Ads, Microsoft Ads tracking
4.  **Organic Search (0/1)**
    -   1 = Visit from non-paid search results
    -   Example: Customer finds site through Google search for brand
        name
    -   Tracked through: Google Analytics source/medium
5.  **Direct (0/1)**
    -   1 = Direct URL entry or bookmark use
    -   Example: Customer types website address directly
    -   Tracked through: Google Analytics

### Data Integration Challenges

Fashion retailers typically face several challenges when collecting this
data:

1.  **Technical Limitations**
    -   Cookie blocking and ITP (Intelligent Tracking Prevention)
    -   Mobile app vs. web tracking differences
    -   Cross-device customer journeys
2.  **Privacy Regulations**
    -   GDPR compliance in Europe
    -   CCPA compliance in California
    -   Cookie consent requirements

### Typical Technology Stack

Most fashion e-commerce companies use a combination of tools:

1.  **Core Platforms**
    -   Analytics: Google Analytics 4
    -   E-commerce: Shopify Plus
    -   Customer Data Platform: Segment
2.  **Marketing Tools**
    -   Email platforms
    -   Social media management
    -   Ad platforms

### Data Collection Example

Here’s how a typical customer journey might be tracked:

In [ ]:
# Example customer journey data structure
customer_journey = {
    'customer_id': '12345',
    'touchpoints': [
        {
            'channel': 'Instagram',
            'timestamp': '2024-02-01 10:30:00',
            'type': 'post_click',
            'utm_source': 'instagram',
            'utm_medium': 'social',
            'utm_campaign': 'spring_collection'
        },
        {
            'channel': 'Email',
            'timestamp': '2024-02-02 14:20:00',
            'type': 'email_open',
            'campaign_id': 'new_arrival_alert'
        },
        {
            'channel': 'Direct',
            'timestamp': '2024-02-02 14:25:00',
            'type': 'website_visit'
        }
    ],
    'conversion': {
        'occurred': True,
        'timestamp': '2024-02-02 14:40:00',
        'value': 150.00,
        'products': ['dress_123', 'accessory_456']
    }
}

The dataset we’ll analyze contains 10,000 customer journeys from the
Spring/Summer 2024 collection launch, tracking whether customers were
exposed to each marketing channel and if they ultimately made a
purchase.

## Creating the Dataset

First, let’s generate a synthetic dataset that mimics realistic fashion
e-commerce conversion rates:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

# Set random seed for reproducibility
np.random.seed(42)

# Generate synthetic dataset with 10,000 samples (more realistic sample size)
n_samples = 10000
data = {
    "Email": np.random.choice([0, 1], size=n_samples, p=[0.7, 0.3]),  # 30% email exposure
    "Social Media": np.random.choice([0, 1], size=n_samples, p=[0.4, 0.6]),  # 60% social media exposure
    "Paid Search": np.random.choice([0, 1], size=n_samples, p=[0.8, 0.2]),  # 20% paid search exposure
    "Organic Search": np.random.choice([0, 1], size=n_samples, p=[0.7, 0.3]),  # 30% organic search exposure
    "Direct": np.random.choice([0, 1], size=n_samples, p=[0.85, 0.15]),  # 15% direct traffic
}

# Set true influence of each channel (coefficients calibrated for ~2% conversion rate)
true_coeffs = np.array([0.5, 0.8, 0.3, 0.2, 0.4])  # Social Media still has highest impact
intercept = -4.5  # Much lower intercept to achieve realistic conversion rate

# Calculate conversion probabilities
X = np.column_stack([data[key] for key in data])
logits = intercept + np.dot(X, true_coeffs)
probabilities = 1 / (1 + np.exp(-logits))
conversions = np.random.binomial(1, probabilities)

# Create final DataFrame
df = pd.DataFrame(data)
df["Converted"] = conversions

# Display conversion rate and channel exposure statistics with detailed explanations
print("\n=== Understanding Exposure and Conversion Rates ===\n")
print("Overall Conversion Rate Analysis:")
conversion_rate = df["Converted"].mean() * 100
print(f"Overall conversion rate: {conversion_rate:.2f}%")
print(f"This means out of all {n_samples} customer journeys, about {int(n_samples * conversion_rate/100)} resulted in a purchase.")

print("\nChannel-Specific Analysis:")
for channel in data.keys():
    exposed_count = df[channel].sum()
    exposed_conv_rate = df[df[channel] == 1]["Converted"].mean() * 100
    
    print(f"\n{channel}:")
    print(f"Exposure Rate: {exposed_count/n_samples*100:.1f}%")
    print(f"Conversion Rate When Exposed: {exposed_conv_rate:.2f}%")
    print(f"Number of Customers Exposed: {exposed_count}")
    
# Create visualization comparing exposure vs conversion rates
plt.figure(figsize=(10,6))
channels = list(data.keys())
exposure_rates = [df[channel].mean() * 100 for channel in channels]
conversion_rates = [df[df[channel] == 1]["Converted"].mean() * 100 for channel in channels]

x = np.arange(len(channels))
width = 0.35

fig, ax = plt.subplots(figsize=(12, 6))
rects1 = ax.bar(x - width/2, exposure_rates, width, label='Exposure Rate', color='plum')
rects2 = ax.bar(x + width/2, conversion_rates, width, label='Conversion Rate When Exposed', color='purple')

ax.set_ylabel('Percentage (%)')
ax.set_title('Exposure Rates vs Conversion Rates by Channel')
ax.set_xticks(x)
ax.set_xticklabels(channels)
ax.legend()

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Understanding Exposure and Conversion Rates

In marketing analytics, we track two crucial metrics: exposure rates and
conversion rates. Let’s understand what these mean and why they matter
for fashion retail:

### Exposure Rates

Exposure rates tell us what percentage of ALL customers encountered each
marketing channel. For example:  
- If Social Media has a 60% exposure rate, this means 6,000 out of
10,000 customers saw social media content - If Email has a 30% exposure
rate, this means 3,000 customers saw emails

This helps retailers understand:  
- The reach of each marketing channel - Which channels are most visible
to customers - Where there might be opportunities to expand reach

### Conversion Rates

We look at conversion rates in two ways:

1.  **Overall Conversion Rate** (~2%):
    -   This is the percentage of ALL customers who made a purchase
    -   In our data, about 200 out of 10,000 customers bought something
    -   This serves as our baseline for comparison
2.  **Channel-Specific Conversion Rates**:
    -   The percentage of people who purchased after seeing each
        specific channel
    -   If a channel’s conversion rate is higher than the overall 2%,
        it’s performing above average

### The Relationship Between Exposure and Conversion

Understanding this relationship helps identify channel effectiveness:

-   **High Exposure + Low Conversion**
    -   Example: Social media might reach 60% of people but convert at
        only 2.5%
    -   This suggests good awareness but possibly weak call-to-action
    -   Strategy: Improve conversion optimization for this channel
-   **Low Exposure + High Conversion**
    -   Example: Email might reach only 30% but convert at 4%
    -   This suggests a highly effective channel with room for growth
    -   Strategy: Focus on growing email list size

### Using These Insights

Fashion retailers can use this information to:

1.  Allocate marketing budgets more effectively
2.  Identify which channels need optimization
3.  Find opportunities for expanding successful channels
4.  Make data-driven decisions about marketing strategy

## Logistic Regression Analysis

Now, let’s analyze this more realistic dataset using logistic regression
to understand the impact of each marketing channel:

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt

# Split data into features (X) and target (y)
X = df.drop(columns=["Converted"])
y = df["Converted"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Predict and evaluate accuracy
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

# Get feature coefficients
coefficients = model.coef_[0]
feature_importance = pd.DataFrame({
    "Channel": X.columns,
    "Coefficient": coefficients,
    "Odds Ratio": np.exp(coefficients)
})
feature_importance["Relative Importance (%)"] = (abs(feature_importance["Coefficient"]) / 
                                               abs(feature_importance["Coefficient"]).sum()) * 100
feature_importance = feature_importance.sort_values(by="Relative Importance (%)", ascending=False)

# Display detailed results
print("\nChannel Importance Analysis:")
print(feature_importance)
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\n=== Understanding the Metrics ===")
print("\n1. Coefficients:")
print("- These are the weights our model assigns to each marketing channel")
print("- Positive coefficients mean the channel increases conversion probability")
print("- Negative coefficients mean the channel decreases conversion probability")
print("- The larger the coefficient (in absolute value), the stronger the effect")
print(f"Example: If social media has a coefficient of {feature_importance.loc[feature_importance['Channel'] == 'Social Media', 'Coefficient'].values[0]:.2f}, ")
print("  This means having social media exposure increases the log-odds of conversion by that amount")

print("\n2. Odds Ratio:")
print("- This is easier to interpret than coefficients")
print("- Shows how much the odds of conversion multiply when exposed to a channel")
print("- Odds ratio > 1: increases chances of conversion")
print("- Odds ratio < 1: decreases chances of conversion")
print(f"Example: If social media has an odds ratio of {feature_importance.loc[feature_importance['Channel'] == 'Social Media', 'Odds Ratio'].values[0]:.2f}:")
print("  Being exposed to social media makes conversion that many times more likely")

print("\n3. Relative Importance (%):")
print("- Shows each channel's contribution relative to all channels")
print("- Calculated as: |channel coefficient| / sum(|all coefficients|) * 100")
print("- Helps compare channels on same scale")
print(f"Example: {feature_importance.iloc[0]['Channel']} has {feature_importance.iloc[0]['Relative Importance (%)']:.1f}% relative importance:")
print("  This means it contributes that percentage of the total predictive power")
print("  Making it the most important channel in our analysis")

# Plot feature importance with fashion-oriented colors
plt.figure(figsize=(10,6))
plt.barh(feature_importance["Channel"], feature_importance["Relative Importance (%)"], 
         color='plum', edgecolor='purple')
plt.xlabel("Relative Importance (%)")
plt.ylabel("Marketing Channel")
plt.title(f"Marketing Channel Impact on Conversion\nOverall Conversion Rate: {conversion_rate:.2f}%")
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## Interpretation of Results for LuxeStyle Boutique

Our analysis of LuxeStyle’s marketing data now reflects realistic
e-commerce conversion rates and reveals several key insights:

1.  **Overall Conversion Rate**: The simulated data shows a conversion
    rate of approximately 2%, which aligns with typical fashion
    e-commerce benchmarks. This means that out of every 100 customer
    journeys, about 2 result in a purchase.

2.  **Channel Impact Analysis**:

    -   **Social Media Dominance**: Despite the low overall conversion
        rate, social media shows the strongest relative influence on
        increasing conversion probability
    -   **Email Performance**: Email marketing demonstrates the
        second-highest impact, particularly effective for retargeting
        and promotional campaigns
    -   **Direct Traffic**: Brand recognition and influencer
        partnerships show moderate impact
    -   **Search Channels**: Both paid and organic search contribute to
        conversions but with lower relative importance

3.  **Channel Exposure Analysis**:

    -   Social media has the highest exposure rate (60% of customer
        journeys)
    -   Direct traffic has the lowest exposure rate (15%) but good
        conversion impact
    -   Email reaches 30% of potential customers

### Recommendations for LuxeStyle

Based on these more realistic findings, we recommend:

1.  **Social Media Strategy**:
    -   Continue strong investment in social media, focusing on
        conversion optimization
    -   Develop more shoppable posts to reduce friction in the purchase
        journey
    -   Test different content types to identify what drives highest
        conversion rates
2.  **Email Marketing Optimization**:
    -   Focus on growing the email list given its strong conversion
        impact
    -   Implement abandoned cart and browse recovery emails
    -   Test timing and frequency of emails to maximize engagement
3.  **Multi-Channel Approach**:
    -   While maintaining focus on high-performing channels, keep a
        balanced presence across all channels
    -   Develop channel-specific content strategies
    -   Track and optimize cross-channel customer journeys
4.  **Conversion Rate Optimization**:
    -   Given the realistic 2% conversion rate, focus on both:
        1.  Increasing traffic from high-converting channels
        2.  Optimizing the conversion funnel to improve rates across all
            channels

This analysis provides LuxeStyle with actionable insights based on
realistic e-commerce metrics, helping them optimize their marketing
strategy in the competitive fashion space.